# Lesson 25: Integrating Multiple AI Agents with MCP

Throughout the course, we've built two agents: Nova for research and Brown for writing. Now it's time to integrate them into a unified system. In this lesson, we will explore how to use both the Nova research agent and the Brown writing workflow together by leveraging the Model Context Protocol (MCP).

We've already seen how each agent works as an MCP server in previous lessons. Nova exposes 11 tools for research tasks, while Brown provides 3 tools for article generation and editing. The beauty of MCP is that it makes integration straightforward. We'll explore two approaches:

1. **Multi-Server MCP Client**: A single MCP client that connects to multiple independent MCP servers. To run both agents together, you execute their MCP prompts in sequence.
2. **Composed MCP Server**: A single MCP client that connects to a single MCP server that composes multiple MCP servers together using FastMCP's composition features. The composed server can add new capabilities—like a combined workflow prompt that orchestrates both agents with a single command.

Both approaches have their use cases, and by the end of this lesson, you'll understand when to use each one.

Learning objectives:
- Learn how to connect an MCP client to multiple MCP servers simultaneously
- Understand how to use FastMCP's composition features to create a unified MCP server
- See how composed servers can add orchestration prompts that coordinate multiple agents
- Compare multi-server client vs composed server approaches
- See the practical benefits of MCP for agent integration

## 1. Setup

First, we define some standard Magic Python commands to autoreload Python packages whenever they change:

In [ ]:
%load_ext autoreload
%autoreload 2

### Import Key Packages

In [ ]:
import nest_asyncio
nest_asyncio.apply() # Allow nested async usage in notebooks

## 2. Understanding Multi-Agent Orchestration: The MCP Approach

Before we dive into the technical implementation, let's understand the orchestration model we're using and why it matters.

### The Central LLM Orchestration Pattern

In this lesson, we're implementing what's known as a **Central LLM Orchestration** pattern. In this approach, a single, central LLM (e.g. the one powering your IDE assistant, like Claude in Cursor) has access to tools from multiple specialized agents. When you give it a task, the LLM dynamically decides which agent's tools to use based on the task requirements, maintaining a single conversation context and orchestrating the workflow by selecting the appropriate tools as needed.

This is fundamentally different from other orchestration patterns. A **Supervisor Agent** pattern would have one agent explicitly delegate entire sub-tasks to worker agents. A **Sequential Pipeline** would force agents to always execute in a fixed order. **Peer-to-Peer Communication** would allow agents to directly message each other. Our central LLM orchestration is simpler: the LLM acts as a intelligent tool selector rather than an explicit coordinator.

We'll learn more about these other patterns in another lesson.

### Why This Pattern Works Well with MCP

The Model Context Protocol makes this orchestration pattern particularly elegant. Both Nova and Brown expose their capabilities as MCP tools with clear descriptions, allowing the central LLM to discover all available tools through a single, standardized protocol. This unified tool interface eliminates the need for custom integration code: because both agents speak MCP, we don't need to write adapters or APIs. The client simply connects to both servers and aggregates their tools.

The LLM's natural reasoning ability handles the orchestration logic. For example, it intuitively understands that it should use Nova's research tools first, review the results, and then use Brown's writing tools with that research as input. There's no need to explicitly program this workflow or create complex state machines.

The flexibility is another key advantage. You can easily add or remove agents by simply updating the configuration file without touching any code. This pattern essentially treats specialized agents as "tool libraries" that a central reasoning engine can draw from as needed, making the system both modular and maintainable.

### The Rationale: Why Choose Central LLM Orchestration?

This orchestration pattern offers several compelling advantages that make it ideal for our use case of integrating Nova and Brown.

The most immediate benefit is **simplicity and maintainability**. All decision-making happens in one place—the central LLM, which means the workflow is transparent and easy to understand. You can see which tools the LLM chooses in real-time as it works through a task. There's no need for complex state management or inter-agent communication protocols, which reduces the cognitive overhead of understanding and debugging the system.

The pattern also enables **natural task decomposition**. The central LLM can break down complex requests on the fly without requiring predefined workflows. Even more importantly, it can adapt its strategy based on intermediate results. For example, if research reveals unexpected information, the LLM can adjust its writing approach accordingly. This adaptive behavior is especially valuable for human-in-the-loop workflows common in IDE environments. You can provide feedback at any point in the process, and the LLM incorporates it naturally without requiring explicit handoff protocols.

The central LLM maintains a **single, unified context window**, which means it can reference information from Nova's research when calling Brown's tools without requiring explicit data passing between agents. This avoids the notorious "siloed knowledge" problem, where critical information gets trapped in one agent's context and becomes unavailable to others who need it.

Perhaps most importantly for our specific use case, this pattern is **perfect for sequential, interdependent tasks**. Our workflow (research followed by writing) is inherently sequential, and the writing task depends heavily on research results. A central orchestrator can easily manage these dependencies because it sees the entire workflow and can make informed decisions about when to transition from research to writing.

### The Tradeoffs: Understanding the Limitations

While central LLM orchestration with MCP is powerful, it's important to understand its limitations and recognize when you might need a different approach.

The first limitation arises from **tool overload**. As the number of available tools grows beyond approximately 15-20, LLMs begin to struggle with reliable tool selection. They may choose suboptimal tools or miss relevant ones entirely. This degradation in performance is well-documented in the research on agent systems. Our system, with 14 tools total, is comfortably within the limit, but if you were to add many more specialized agents, you'd eventually hit this ceiling and need to consider a different pattern.

Another significant limitation is the pattern's inherently **sequential execution model**. If you need to research 50 companies simultaneously, a single LLM executing tools one at a time would be painfully inefficient. In such scenarios, a Supervisor-Worker pattern with parallel execution would be better.

The pattern also struggles with **complex inter-agent dependencies**. If agents need to negotiate with each other, engage in debate, or iteratively refine each other's work through back-and-forth exchanges, direct agent-to-agent communication would be more natural. Our pattern handles simple, linear dependencies well (Nova's output feeds into Brown) but complex multi-way interactions would become awkward and difficult to manage.

However, central LLM orchestration with MCP is the **right default choice** for most agent integration scenarios. It's simple, maintainable, and leverages the LLM's natural reasoning abilities without adding unnecessary complexity. You should only consider more elaborate patterns when you hit clear scaling limits (e.g. too many tools causing selection problems) or have fundamentally different requirements like massive parallelism or complex agent negotiations.

We'll learn more about these other patterns in another lesson.

### Integrating Nova and Brown

In the previous lessons, we built two specialized agents: Nova for research (which ingests article guidelines, performs web research, scrapes sources, and compiles comprehensive research files) and Brown for writing (which takes research and guidelines to generate, review, and edit articles with human-in-the-loop feedback). These agents were designed to work in sequence—Nova gathers the research, and Brown uses that research to write the article—but we've been running them separately. Now we'll orchestrate them together using the central LLM orchestration pattern. Because both agents are already exposed as MCP servers, integration is straightforward: we simply connect to both servers and let the central LLM decide which tools to use and when.

We'll explore two different approaches for achieving this integration.
1. The first is a **Multi-Server MCP Client**, where a single client connects directly to multiple independent MCP servers simultaneously—the client aggregates all tools from both servers and presents them to the LLM.
2. The second is a **Composed MCP Server**, where we create a new server that internally connects to both Nova and Brown, exposing their combined capabilities as a single unified endpoint.

Both approaches achieve the same goal but differ in their deployment and configuration patterns.

## 3. Approach 1: Multi-Server MCP Client

The first approach is to create an MCP client that connects to multiple MCP servers simultaneously. FastMCP's `Client` class supports this out of the box by accepting a configuration object that specifies multiple servers.

### 3.1 Multi-Server Configuration File

Let's look at the configuration file that defines both Nova and Brown servers.

Source: _lessons/25_integrate_agents/mcp_servers_config_http.json_

```json
{
  "mcpServers": {
    "nova-research-agent": {
      "url": "http://localhost:8001/mcp"
    },
    "brown-writing-workflow": {
      "url": "http://localhost:8002/mcp"
    }
  }
}
```

This configuration tells the MCP client how to connect to both servers via HTTP. Each server:
- Has a unique name (`nova-research-agent`, `brown-writing-workflow`)
- Specifies the HTTP URL where the server is listening

### 3.2 Creating the Multi-Server Client

Now let's see how the client code loads this configuration and connects to both servers.

Source: _lessons/agents_integration/mcp_client/src/client.py_

```python
import argparse
import json
from pathlib import Path
from fastmcp import Client

# Parse command line arguments for config file
parser = argparse.ArgumentParser(description="Multi-Server MCP Client")
parser.add_argument("--config", "-c", type=str, default=None,
                    help="Path to MCP servers config file")
args = parser.parse_args()

# Load configuration from JSON file
config_path = Path(args.config) if args.config else Path("mcp_servers_config.json")
with open(config_path) as f:
    config = json.load(f)

server_names = list(config["mcpServers"].keys())
logging.info(f"Found {len(server_names)} MCP servers in configuration: {', '.join(server_names)}")

# Create a single client with multi-server configuration
client = Client(config)

# Connect and fetch capabilities from all servers
async with client:
    tools = await client.list_tools()
    resources = await client.list_resources()
    prompts = await client.list_prompts()
    
    logging.info(
        f"Total capabilities: {len(tools)} tools, {len(resources)} resources, {len(prompts)} prompts"
    )
    
    # Main conversation loop
    while True:
        user_input = input("👤 You: ").strip()
        parsed_input = parse_user_input(user_input)
        
        # Handle the user message (commands, prompts, or normal messages)
        should_continue, thinking_enabled = await handle_user_message(
            parsed_input=parsed_input,
            tools=tools,
            resources=resources,
            prompts=prompts,
            conversation_history=conversation_history,
            mcp_client=client,
            thinking_enabled=thinking_enabled,
            server_names=server_names,
        )
        
        if not should_continue:
            break
```

The key insight here is that `Client(config)` accepts a multi-server configuration. When you call `list_tools()`, `list_resources()`, or `list_prompts()`, the client automatically aggregates capabilities from all connected servers.

The client supports several interactive commands:
- `/tools`, `/resources`, `/prompts` - List available capabilities
- `/prompt/<name>?arg=value` - Load and execute an MCP prompt with optional arguments
- `/resource/<uri>` - Read a resource's content
- `/model-thinking-switch` - Toggle the LLM's thinking mode
- `/quit` - Exit the client

When a prompt is loaded via `/prompt/<name>`, the client retrieves its content from the MCP server and sends it to the LLM, which then executes the workflow using the available tools.

### 3.3 How Capabilities Are Named

When you have multiple servers, how do you distinguish which tool belongs to which server? FastMCP handles this automatically by prefixing the capability names with the server name from your configuration.

For example, with our configuration having `nova-research-agent` and `brown-writing-workflow` as server names:
- Nova's `extract_guidelines_urls` tool becomes `nova-research-agent_extract_guidelines_urls`
- Brown's `generate_article` tool becomes `brown-writing-workflow_generate_article`
- Nova's `full_research_instructions_prompt` prompt becomes `nova-research-agent_full_research_instructions_prompt`
- Brown's `generate_article_prompt` prompt becomes `brown-writing-workflow_generate_article_prompt`

This naming convention makes it easy to identify which capability comes from which server, and ensures there are no naming collisions if both servers happen to expose capabilities with the same base name.

### 3.4 Running the Multi-Server Client

When you run the multi-server client, you'll see output like this:

```bash
$ cd agents_integration/mcp_client
$ uv run -m src.client --config ../../25_integrate_agents/mcp_servers_config_http.json
```

```terminal
INFO:root:Loading MCP server configuration from: mcp_servers_config_http.json
INFO:root:Found 2 MCP servers in configuration: nova-research-agent, brown-writing-workflow
INFO:root:Connecting to MCP servers...
INFO:root:Fetching capabilities from all servers...
INFO:root:Total capabilities: 14 tools, 4 resources, 4 prompts

============================================================
All Servers
============================================================

  - 14 tools available
  - 4 resources available
  - 4 prompts available

Available Commands: /tools, /resources, /prompts, /prompt/<name>?arg=value, /resource/<uri>, /model-thinking-switch, /quit

👤 You:
```

When you type `/prompts`, you'll see all prompts from both servers:

```terminal
👤 You: /prompts
============================================================
Prompts (4)
============================================================

1. nova-research-agent_full_research_instructions_prompt
   Complete Nova research agent workflow instructions.

2. brown-writing-workflow_generate_article_prompt
   Retrieve a prompt that will trigger the article generation workflow using Brown.

3. brown-writing-workflow_edit_article_prompt
   Retrieve a prompt that will trigger the article editing workflow using Brown.

4. brown-writing-workflow_edit_selected_text_prompt
   Retrieve a prompt that will trigger the selected text editing workflow using Brown.

```

This demonstrates that the client successfully connected to both servers and aggregated their capabilities. Notice that each prompt is prefixed with its server name, making it clear which agent provides each capability.

### 3.5 Running the Complete Workflow: Two Prompts in Sequence

This section assumes the client is connected using the configuration from `lessons/25_integrate_agents/mcp_servers_config_http.json`.

To run both agents together—first Nova for research, then Brown for article generation—you need to execute their prompts in sequence.

**Step 1: Run Nova's Research Workflow**

First, trigger Nova's research prompt:

```terminal
👤 You: /prompt/nova-research-agent_full_research_instructions_prompt
```

The LLM receives the prompt instructions and responds:

```terminal
💬 LLM Response: Hello! I will help you execute the Nova research agent workflow. Here are the steps involved:

1.  **Setup:** Extract URLs and local file references from your article guidelines.
2.  **Process Resources:** Concurrently process local files, scrape other web URLs, process GitHub URLs, and transcribe YouTube URLs found in the guidelines.
3.  **Research Loop:** Conduct 3 rounds of web research using Perplexity to generate new queries and gather results.
4.  **Filter Results:** Evaluate and select high-quality sources from the Perplexity research.
5.  **Scrape Research Sources:** Identify and scrape the full content of the most valuable research sources.
6.  **Final Research File:** Compile all gathered and processed research into a comprehensive markdown file.

To begin, please provide the path to your research directory. Also, let me know if you need any modifications to this workflow, such as starting from a specific step or adding user feedback.

👤 You:
```

You then provide the research directory path and let Nova complete the research workflow.

**Step 2: Run Brown's Article Generation Workflow**

Once Nova finishes, you trigger Brown's article generation prompt with the same directory:

```terminal
👤 You: /prompt/brown-writing-workflow_generate_article_prompt?dir_path=/path/to/your/research/directory
```

The LLM receives Brown's prompt and calls the article generation tool:

```terminal
🔧 Function Call (Tool):
  Tool: brown-writing-workflow_generate_article
  Arguments: {
    "dir_path": "/path/to/your/research/directory"
  }

⚡ Executing tool 'brown-writing-workflow_generate_article' via MCP server...
```

Brown then generates the article using the research files Nova created.

### 3.6 The Limitation: No Single Unified Command

While this two-step approach works, it has a significant limitation: **you must manually run two separate prompts in the correct order**. There's no single command that orchestrates the entire research-to-article workflow.

You might think: "Why not just add a combined prompt to one of the servers?" But this creates a design problem:

1. **Adding to Nova?** Nova is a research agent. It shouldn't need to know about Brown's article generation internals. This would create tight coupling between agents.

2. **Adding to Brown?** Similarly, Brown is a writing workflow. It shouldn't need to know how Nova's research steps work.

3. **Writing a manual prompt?** You could craft a long prompt that describes both workflows, but this duplicates the logic already encoded in each agent's prompts. It's error-prone and hard to maintain.

**The clean solution is Approach 2**: create a composed MCP server that proxies both Nova and Brown. This composed server can own the combined workflow prompt without polluting either agent's codebase. The orchestration logic lives in the right place—the integration layer—not in the individual agents.


### 3.7 Running the Multi-Server Client in the Notebook

Now let's run the multi-server client directly from this notebook. The client will connect to both Nova and Brown servers and display their capabilities.

In this setup, we use HTTP transport (Streamable HTTP) for communication between the MCP client and servers. The first code cell starts Nova and Brown as HTTP servers on ports 8001 and 8002 respectively. The second code cell runs the client, which connects to these servers via HTTP URLs defined in `mcp_servers_config_http.json`.

*Note*: The client is interactive, so you can type commands like `/tools`, `/resources`, `/prompts`, or `/quit` when prompted. Type `/quit` to exit the client.

In [ ]:
import subprocess
import time

print("Starting MCP servers as HTTP services...")

# Start Nova MCP server on port 8001
nova_proc = subprocess.Popen([
    "uv", "--directory", "/path/to/course-ai-agents/lessons/research_agent_part_2/mcp_server",
    "run", "-m", "src.server", "--transport", "streamable-http", "--port", "8001"
], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

# Start Brown MCP server on port 8002
brown_proc = subprocess.Popen([
    "uv", "--directory", "/path/to/course-ai-agents/lessons/writing_workflow",
    "run", "-m", "brown.mcp.server", "--transport", "streamable-http", "--port", "8002"
], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

print("Waiting for servers to start...")
time.sleep(10)
print("Servers should be running on ports 8001 (Nova) and 8002 (Brown)")

In [ ]:
# Run the MCP client
import sys
from agents_integration.mcp_client.src.client import main as client_main


async def run_client():
    _argv_backup = sys.argv[:]
    sys.argv = ["client", "--config", "mcp_servers_config_http.json"]
    try:
        await client_main()
    finally:
        sys.argv = _argv_backup

await run_client()

In [ ]:
print("Stopping MCP servers...")
try:
    nova_proc.terminate()
    brown_proc.terminate()
    nova_proc.wait(timeout=5)
    brown_proc.wait(timeout=5)
    print("Servers stopped")
except Exception as e:
    print(f"Error stopping servers: {e}")

After running this cell, you should see:
1. Both servers starting up (Nova Research MCP Server and Brown MCP Server)
2. The total capabilities summary (14 tools, 4 resources, 4 prompts)
3. A welcome message showing all combined capabilities
4. An interactive prompt where you can type commands

Try typing:
- `/prompts` to see all prompts from both servers
- `/prompt/nova-research-agent_full_research_instructions_prompt` to start the research workflow
- `/prompt/brown-writing-workflow_generate_article_prompt?dir_path=/path/to/dir` to generate an article
- `/tools` to see all tools from both servers
- `/resources` to see all resources
- `/quit` to exit

## 4. Approach 2: Composed MCP Server

The second approach is to create a new MCP server that composes the Nova and Brown servers together. Instead of the client connecting to multiple servers, you create a single composed server that internally proxies requests to the underlying servers.

This approach is useful when you want to:
- Package multiple agents as a single deployable unit
- Simplify the client-side configuration (client only needs to know about one server)
- Add a layer of coordination or orchestration between agents

### 4.1 Server Composition Configuration

First, we define which servers to compose. The composed server will connect to Nova and Brown via HTTP:

Source: _lessons/25_integrate_agents/mcp_servers_config_http.json_

```json
{
  "mcpServers": {
    "nova-research-agent": {
      "url": "http://localhost:8001/mcp"
    },
    "brown-writing-workflow": {
      "url": "http://localhost:8002/mcp"
    }
  }
}
```

This is identical to the multi-server client config, but it's used differently: here it defines the servers the composed server will proxy to via HTTP.


### 4.2 Creating the Composed Server

Now let's see how to create a composed server using FastMCP's composition features.

Source: _lessons/agents_integration/mcp_server/src/main.py_

```python
import json
import logging
from pathlib import Path
from fastmcp import Client, FastMCP

def load_server_config() -> dict:
    """Load the MCP servers configuration from JSON file."""
    config_path = Path(__file__).parent.parent / "mcp_servers_to_compose.json"
    with open(config_path) as f:
        return json.load(f)

def create_composed_server() -> FastMCP:
    """Create a composed MCP server by mounting Nova and Brown servers."""
    # Create the main composed server
    mcp = FastMCP(
        name="Nova+Brown Composed Server",
        version="0.1.0",
    )
    
    # Load configuration
    config = load_server_config()
    servers_config = config.get("mcpServers", {})
    
    # Create proxies and mount each server
    for server_name, server_config in servers_config.items():
        # Wrap the server config in the structure expected by Client
        client_config = {"mcpServers": {server_name: server_config}}
        
        # Create a client for this server
        client = Client(client_config)
        
        # Create a proxy from the client
        proxy = FastMCP.as_proxy(client)
        
        # Mount the proxy WITHOUT a prefix - capabilities keep their original names
        mcp.mount(proxy)
    
    # Register the combined workflow prompt
    register_combined_prompt(mcp)
    
    return mcp

def register_combined_prompt(mcp: FastMCP) -> None:
    '''Register the combined research and writing workflow prompt.'''
    
    @mcp.prompt()
    def full_research_and_writing_workflow(dir_path: Path) -> str:
        """Complete workflow for research and article generation.
        
        This prompt combines the Nova research agent workflow with the Brown
        article generation workflow, providing end-to-end instructions for
        conducting comprehensive research and generating an article from that research.
        
        Args:
            dir_path: Path to the directory that will contain research resources
                     and the final article.
        
        Returns:
            A formatted prompt string with complete workflow instructions.
        """
        return f"""
# Complete Research and Article Generation Workflow

This workflow combines two phases: research (Nova) and article generation (Brown).

## PHASE 1: Research (Nova)

Your job is to execute the workflow below...
[Full Nova workflow instructions]

## PHASE 2: Article Generation (Brown)

Once the research phase is complete, use Brown to generate an article
using the research from: {dir_path}
""".strip()

if __name__ == "__main__":
    composed_server = create_composed_server()
    composed_server.run()
```

Let's break down the key steps:

1. First we create a FastMCP instance. This is our composed server.
2. For each server to compose:
   - Create a `Client` that connects to that server
   - Use `FastMCP.as_proxy(client)` to create a proxy object
   - Use `mcp.mount(proxy)` to mount it **without a prefix**. When you mount a proxy without a prefix, all capabilities keep their original names. This means tools, resources, and prompts from both Nova and Brown are exposed exactly as they were defined in each agent.
3. After mounting both servers, we add a new capability: the `full_research_and_writing_workflow` prompt. This is the key advantage of the composed server approach—we can add orchestration logic that coordinates both agents without modifying either agent's code. The `@mcp.prompt()` decorator registers a new MCP prompt that combines instructions for both Nova's research workflow and Brown's article generation. When a user triggers this prompt, the LLM receives comprehensive instructions to run both agents end-to-end in a single conversation.

### 4.3 Running the Composed Server

To use the composed server, you need a client config that points to it:

Source: _lessons/25_integrate_agents/mcp_composed_server_config_http.json_

```json
{
  "mcpServers": {
    "nova-brown-composed": {
      "url": "http://localhost:8003/mcp"
    }
  }
}
```

Now when you run the client with this config:

```bash
$ cd agents_integration/mcp_client
$ uv run -m src.client --config ../../25_integrate_agents/mcp_composed_server_config_http.json
```

You'll see:

```terminal
INFO:root:Loading MCP server configuration from: mcp_composed_server_config_http.json
INFO:root:Found 1 MCP servers in configuration: nova-brown-composed
INFO:root:Connecting to MCP servers...
INFO:__main__:Starting composed MCP server...
INFO:__main__:Loading server configuration...
INFO:__main__:Found 2 servers to compose: ['nova-research-agent', 'brown-writing-workflow']
INFO:__main__:Creating proxy for nova-research-agent...
INFO:__main__:Mounting nova-research-agent without prefix...
INFO:__main__:Creating proxy for brown-writing-workflow...
INFO:__main__:Mounting brown-writing-workflow without prefix...
INFO:__main__:Composed server created successfully!
INFO:__main__:Running composed server...

INFO:root:Fetching capabilities from all servers...
INFO:root:Total capabilities: 14 tools, 4 resources, 5 prompts

============================================================
All Servers
============================================================

  - 14 tools available
  - 4 resources available
  - 5 prompts available

Available Commands: /tools, /resources, /prompts, /prompt/<name>?arg=value, /resource/<uri>, /model-thinking-switch, /quit

👤 You:
```

Notice that:
- The composed server mounts both agents **without prefixes**, so capabilities keep their original names
- There are now **5 prompts** (not 4)—the composed server has added a new `full_research_and_writing_workflow` prompt
- The client sees a single unified interface with all capabilities from both agents plus the new orchestration prompt

### 4.4 Listing Prompts from the Composed Server

When you type `/prompts`, you'll see all 5 prompts available:

```terminal
👤 You: /prompts
============================================================
Prompts (5)
============================================================

1. full_research_and_writing_workflow
   Complete workflow for research and article generation.

   This prompt combines the Nova research agent workflow with the Brown
   article generation workflow, providing end-to-end instructions for
   conducting comprehensive research and generating an article from that research.

2. full_research_instructions_prompt
   Complete Nova research agent workflow instructions.

3. generate_article_prompt
   Retrieve a prompt that will trigger the article generation workflow using Brown.

4. edit_article_prompt
   Retrieve a prompt that will trigger the article editing workflow using Brown.

5. edit_selected_text_prompt
   Retrieve a prompt that will trigger the selected text editing workflow using Brown.
```

Notice that `full_research_and_writing_workflow` is **new**—it's not from Nova or Brown, but added by the composed server itself. This prompt orchestrates both agents, instructing the LLM to:
1. Run the complete Nova research workflow
2. Then use Brown to generate an article from that research

This is the key advantage of the composed server approach: you can add coordination logic at the integration layer without modifying the individual agents.

### 4.5 Running the Complete Workflow with One Prompt

Now let's see the power of the composed server approach. Instead of running two separate prompts like in Approach 1, we can trigger the complete end-to-end workflow with a single command:

```terminal
👤 You: /prompt/full_research_and_writing_workflow?dir_path=/absolute/path/to/research_folder
```

The LLM receives the combined prompt and immediately understands it needs to execute both phases:

```terminal
🤔 LLM's Thoughts:
Okay, I'm looking at a two-phase operation here: Research (Nova) and Article Generation (Brown).
This is a well-defined process, and I need to execute it methodically. The first phase, Nova, 
is the more involved one, and it's where my focus will be initially...

🔧 Function Call (Tool):
  Tool: extract_guidelines_urls
  Arguments: {
    "research_directory": "/absolute/path/to/research_folder"
  }

⚡ Executing tool 'extract_guidelines_urls' via MCP server...
✅ Tool execution successful!

🔧 Function Call (Tool):
  Tool: process_local_files
  Arguments: {
    "research_directory": "/absolute/path/to/research_folder"
  }

⚡ Executing tool 'process_local_files' via MCP server...
✅ Tool execution successful!

🔧 Function Call (Tool):
  Tool: scrape_and_clean_other_urls
  Arguments: {
    "research_directory": "/absolute/path/to/research_folder"
  }

⚡ Executing tool 'scrape_and_clean_other_urls' via MCP server...
```

The LLM continues executing the Nova research workflow tools, then seamlessly transitions to Brown's article generation. **All with a single prompt command.**

This solves the limitation we saw in Approach 1, where you had to manually run two prompts in sequence. The composed server encapsulates the orchestration logic, providing a clean, single-command interface to the complete workflow.

### 4.6 Running the Composed Server Client in the Notebook

Now let's run the client with the composed server configuration. This time, the client connects to a single composed server that internally proxies to both Nova and Brown.

In this setup, we use HTTP transport (Streamable HTTP) for all MCP communication. The first code cell starts three servers:
1. Nova MCP server on port 8001
2. Brown MCP server on port 8002
3. The composed MCP server on port 8003, which connects to Nova and Brown via HTTP

The second code cell runs the MCP client, which connects to the composed server at `http://localhost:8003/mcp` using the configuration in `mcp_composed_server_config_http.json`. The composed server then proxies all requests to Nova and Brown as needed.

*Note*: This is also interactive. Type `/quit` to exit when you're done exploring.

In [ ]:
import subprocess
import time

print("Starting MCP servers as HTTP services...")

# Start Nova MCP server on port 8001
nova_proc = subprocess.Popen([
    "uv", "--directory", "/path/to/course-ai-agents/lessons/research_agent_part_2/mcp_server",
    "run", "-m", "src.server", "--transport", "streamable-http", "--port", "8001"
], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Start Brown MCP server on port 8002
brown_proc = subprocess.Popen([
    "uv", "--directory", "/path/to/course-ai-agents/lessons/writing_workflow",
    "run", "-m", "brown.mcp.server", "--transport", "streamable-http", "--port", "8002"
], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

print("Waiting for Nova and Brown servers to start...")
time.sleep(10)
print("Nova and Brown servers should be running on ports 8001 and 8002")

# Start Composed MCP server on port 8003
# The composed server connects to Nova and Brown via HTTP
composed_proc = subprocess.Popen([
    "uv", "--directory", "/path/to/course-ai-agents/lessons/agents_integration/mcp_server",
    "run", "python", "-m", "src.main",
    "--transport", "streamable-http",
    "--port", "8003",
    "--config", "/path/to/course-ai-agents/lessons/25_integrate_agents/mcp_servers_config_http.json"
], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

print("Waiting for Composed server to start...")
time.sleep(10)
print("Composed server should be running on port 8003")

In [ ]:
import sys
from agents_integration.mcp_client.src.client import main as client_main


async def run_client():
    _argv_backup = sys.argv[:]
    sys.argv = ["client", "--config", "mcp_composed_server_config_http.json"]
    try:
        await client_main()
    finally:
        sys.argv = _argv_backup


# Start client with in-memory server
await run_client()

In [ ]:
print("Stopping MCP servers...")
try:
    nova_proc.terminate()
    brown_proc.terminate()
    composed_proc.terminate()
    nova_proc.wait(timeout=5)
    brown_proc.wait(timeout=5)
    composed_proc.wait(timeout=5)
    print("Servers stopped")
except Exception as e:
    print(f"Error stopping servers: {e}")

After running this cell, you should see:
1. The composed server starting (Nova+Brown Composed Server)
2. Log messages showing the composition process (creating proxies, mounting servers **without prefixes**)
3. Both Nova and Brown servers starting up in the background
4. The total capabilities summary (14 tools, 4 resources, **5 prompts**)
5. A welcome message showing all combined capabilities
6. An interactive prompt for commands

Notice the key differences compared to the multi-server client (Approach 1):
- Only one server name in the config (`nova-brown-composed`)
- Capabilities keep their original names (no prefixes added)
- **5 prompts instead of 4**—the new `full_research_and_writing_workflow` prompt is added by the composed server
- Single command to run both agents end-to-end

Try these commands:
- `/prompts` to see all 5 prompts (including the new combined workflow prompt)
- `/prompt/full_research_and_writing_workflow?dir_path=/path/to/dir` to run both agents with one command
- `/tools` to see all tools from both agents
- `/resources` to see all resources
- `/quit` to exit

## 5. Multi-Server Client vs Composed Server: When to Use Each

Both approaches achieve the same goal: using Nova and Brown together. But they have different use cases and trade-offs.

| Aspect | Multi-Server Client | Composed Server |
|--------|---------------------|-----------------|
| **How it works** | The client connects to multiple independent servers simultaneously | A single server internally proxies to multiple underlying servers |
| **Orchestration prompts** | No built-in way to add combined prompts—must run agent prompts sequentially | Can add prompts that orchestrate multiple agents (e.g., `full_research_and_writing_workflow`) |
| **Single command workflow** | Requires running multiple prompts in sequence (Nova, then Brown) | Single `full_research_and_writing_workflow` prompt runs both agents end-to-end |
| **Client configuration** | Client needs to know about all servers | Single endpoint (simpler client config) |
| **Flexibility** | Easy to add/remove servers without changing code | Requires code changes to modify composition |
| **Use when** | • Developing or testing agents<br>• Quickly combining existing agents<br>• Need flexibility to add/remove agents dynamically | • Deploying agents as a unified system<br>• Need coordination logic between agents<br>• Want simpler client experience<br>• Building a product that packages multiple agents |

## 6. Conclusion

In this lesson, we explored how to integrate the Nova research agent and Brown writing workflow using MCP. We learned:

1. **Multi-Server MCP Client**: How to connect a single client to multiple MCP servers using FastMCP's multi-server configuration. Capabilities are prefixed with server names, and you run agent workflows by triggering their prompts sequentially.
2. **Composed MCP Server**: How to use FastMCP's composition features (`as_proxy()` and `mount()`) to create a unified server that proxies multiple agents. Crucially, composed servers can add new orchestration prompts—like `full_research_and_writing_workflow`—that coordinate multiple agents without modifying their code.
3. **Use Cases**: When to use multi-server client vs composed server. The multi-server approach is better for flexibility and independence, while the composed server is better for having a unified interface, orchestration capabilities, and single-command workflows.

The key insight is that MCP makes agent integration straightforward. Because both Nova and Brown are already MCP servers, we don't need to write custom integration code. We simply leverage MCP's standardized protocol and FastMCP's composition features. The composed server approach goes further by allowing us to add orchestration logic at the integration layer—providing a single-command interface to run both agents end-to-end—without polluting either agent's codebase.

In a real-world scenario, you would use these integrated agents within an IDE like Cursor. You could:
- Configure the composed server in Cursor's MCP settings
- Trigger the `full_research_and_writing_workflow` prompt with a single command
- Watch as the LLM automatically orchestrates Nova's research and Brown's article generation
- Review the research file and generated article
- Iterate on the article with Brown's editing tools
- All from within your editor, with beautiful diff views and human-in-the-loop feedback

In the next lesson, you'll see how to use both agents from Cursor to work on an article end-to-end. In Part 3 of this course, we'll explore production deployment, including how to deploy these composed servers remotely, add monitoring with Opik, and implement security measures.